# <center>Data Wrangling</center>

<img src="../image/wrangling_DALLE.jpeg" width=30% align="right" style="in-line">

>*80% of data wrangling is spent cleaning the data.*
>
>*The other 20% is spent complaining about cleaning the data.*
>
>— ChatGPT 4o

<img src="../image/quote2_ChatGPT.png" width=60% align="left" style="in-line">

## Learning goals

1. Be able to perform web scraping across multiple pages.
2. Be able to load, explore, and understand the structure of datasets using Python.
3. Become familiar with common data wrangling tasks, including handling missing values, merging datasets, filtering, and sorting data.

## Agenda

1. [Review: web scraping](#1)
2. [Data wrangling](#2)

<a name="1"></a>
## Agenda 1. Web scraping

&#x1F4DD; **<font color=dodgerblue>FROM LAST WEEK: </font>** Here is an example project. We would like to find out what the European Union has been doing to advance sustainable mobility and transport. One possible data source is the news (https://transport.ec.europa.eu/news-events/news_en) we just scraped, but we need more information other than the title of the news.

So now please write some code to collect **the date, the title, the short description, the news type, and the link to the full text** of all news. Save the data to a **csv** file.

Here are some tips:
1. How many pages do you need to scrape? Observe how the web addresses change between the first page and the second.
2. Remember we have talked about **avoid overloading servers** in ethics. Make sure to use `time.sleep()`.
3. Maybe AI tools such as ChatGPT can help. But you need to make sure its solution works.

If you would like to challenge yourself, see if you can scrape the full text (not the short description) of the news. Try with one or two pieces of news would be enough.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

### Step 1: Send an HTTP request

In [ ]:
url = "https://transport.ec.europa.eu/news-events/news_en?page=0"
page = requests.get(url)

In [ ]:
print(page)

### Step 2: Parse the HTML content

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
print(soup)
#print(soup.prettify())

### Step 3: Locate the HTML elements containing the desired data

Chrome: `View` -> `Developer` -> `Inspect Elements`

<img src="../image/news_elements.png">

In [ ]:
news = soup.find_all("article", class_="ecl-content-item")

In [ ]:
print(news)

In [ ]:
len(news)

### Step 4: Extract data

In [ ]:
for item in news:
    date = item.time.attrs['datetime']
    title = item.find("a", class_="ecl-link ecl-link--standalone").get_text()
    desc = item.find("div", class_="ecl-content-block__description").get_text()
    news_type = item.find("li", class_="ecl-content-block__primary-meta-item").get_text()
    link = item.find("a", class_="ecl-link ecl-link--standalone", href=True)["href"]
    print(date,title,desc)
    print(news_type)
    print(link)
    print("=====")

### Save data to csv

To scrape multiple pages, we need to observe how the web addresses change.

In [ ]:
#page_number = range(7,18) # year 2023. your number could be different
page_number = range(0,8) # year 2024
stem = "https://transport.ec.europa.eu/news-events/news_en?page="

In [ ]:
#csv_filename = "news2023.csv" # give the csv file a name
csv_filename = "news2024.csv"

In [ ]:
for i in page_number: # loop over all the pages
    url = stem + str(i)
    print(url)
    
    page = requests.get(url) # send the request
    soup = BeautifulSoup(page.content, 'html.parser') # parse the content
    news = soup.find_all("article", class_="ecl-content-item") # locate all the news
    
    for item in news: # loop over each article
        try: # use try except to skip error e.g., NoneType object - empty elements
            date = item.time.attrs['datetime']
        except (AttributeError, KeyError, TypeError) as e:
            date = ""
            print(f"Error extracting date: {e}")
            print(item)
            print("====")
        try:
            title = item.find("a", class_="ecl-link ecl-link--standalone").get_text()
        except AttributeError as e:
            title = ""
            print(f"Error extracting title: {e}")
            print(item)
            print("====")
        try:
            desc = item.find("div", class_="ecl-content-block__description").get_text()
        except AttributeError as e:
            desc = ""
            print(f"Error extracting description: {e}")
            print(item)
            print("====")
        try:
            news_type = item.find("li", class_="ecl-content-block__primary-meta-item").get_text()
        except AttributeError as e:
            news_type = ""
            print(f"Error extracting news type: {e}")
            print(item)
            print("====")
        try:
            link = item.find("a", class_="ecl-link ecl-link--standalone", href=True)["href"]
        except (AttributeError, TypeError) as e:
            link = ""
            print(f"Error extracting link: {e}")
            print(item)
            print("====")
        csv.writer(open(csv_filename, "a")).writerow([date, title, desc, news_type, link])
        
    time.sleep(2)

I got 8 errors running the above cell (for year 2023). It is okay. I have saved the errors in case we need to go back.

<a name="error"></a>
The error output

```
https://transport.ec.europa.eu/news-events/news_en?page=7
https://transport.ec.europa.eu/news-events/news_en?page=8
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-12-12T12:00:00Z">12 December 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.wbif.eu/news-details/transport-ministers-discuss-reforms-within-western-balkans-transport-community-summit"><span class="ecl-link__label">Transport Ministers discuss reforms within Western Balkans at Transport Community Summit</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>Transport ministers from the Western Balkans, Georgia, Moldova, and Ukraine, as well as the European Commission, gathered at the annual Ministerial Council Meeting of the Transport Community in Skopje, North Macedonia, on 12 December 2023.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-12-12T12:00:00Z">12 December 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.wbif.eu/news-details/transport-ministers-discuss-reforms-within-western-balkans-transport-community-summit"><span class="ecl-link__label">Transport Ministers discuss reforms within Western Balkans at Transport Community Summit</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>Transport ministers from the Western Balkans, Georgia, Moldova, and Ukraine, as well as the European Commission, gathered at the annual Ministerial Council Meeting of the Transport Community in Skopje, North Macedonia, on 12 December 2023.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=9
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-10-25T12:00:00Z">25 October 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.ecac-ceac.org/news/960-press-release-european-aviation-leaders-unite-for-holistic-sustainability-in-the-ecac-eu-dialogue"><span class="ecl-link__label">European Commission joins the 12th edition of the ECAC/EU Dialogue</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>European aviation leaders have come together in Valencia for the 12th edition of the ECAC/EU Dialogue, reaffirming their commitment to sustainable aviation.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-10-25T12:00:00Z">25 October 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.ecac-ceac.org/news/960-press-release-european-aviation-leaders-unite-for-holistic-sustainability-in-the-ecac-eu-dialogue"><span class="ecl-link__label">European Commission joins the 12th edition of the ECAC/EU Dialogue</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>European aviation leaders have come together in Valencia for the 12th edition of the ECAC/EU Dialogue, reaffirming their commitment to sustainable aviation.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=10
https://transport.ec.europa.eu/news-events/news_en?page=11
https://transport.ec.europa.eu/news-events/news_en?page=12
https://transport.ec.europa.eu/news-events/news_en?page=13
https://transport.ec.europa.eu/news-events/news_en?page=14
https://transport.ec.europa.eu/news-events/news_en?page=15
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-03-14T12:00:00Z">14 March 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.eib.org/en/press/all/2023-108-investeu-eur3-4-billion-to-modernise-the-palermo-catania-railway-line"><span class="ecl-link__label">Italy: InvestEU - €3.4 billion to modernise the Palermo-Catania railway line</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The modernisation of 178 km of the Palermo-Catania line will reduce current travel times by a third, linking the two cities with a direct two-hour rail service, which will have a significant impact on economic, social, and sustainable development in Sicily.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-03-14T12:00:00Z">14 March 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.eib.org/en/press/all/2023-108-investeu-eur3-4-billion-to-modernise-the-palermo-catania-railway-line"><span class="ecl-link__label">Italy: InvestEU - €3.4 billion to modernise the Palermo-Catania railway line</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The modernisation of 178 km of the Palermo-Catania line will reduce current travel times by a third, linking the two cities with a direct two-hour rail service, which will have a significant impact on economic, social, and sustainable development in Sicily.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-02-28T12:00:00Z">28 February 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://mobilityweek.eu/media-corner/"><span class="ecl-link__label">Braga, Sofia and Zagreb among the finalists for European urban mobility awards</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The finalists for the EUROPEANMOBILITYWEEK award 2022 and the first-ever MOBILITYACTION award have been announced.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-02-28T12:00:00Z">28 February 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://mobilityweek.eu/media-corner/"><span class="ecl-link__label">Braga, Sofia and Zagreb among the finalists for European urban mobility awards</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The finalists for the EUROPEANMOBILITYWEEK award 2022 and the first-ever MOBILITYACTION award have been announced.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=16
https://transport.ec.europa.eu/news-events/news_en?page=17
```

<a name="2"></a>
## Agenda 2. Data wrangling

Data wrangling is the process of converting raw data into a usable form. It typically involves examining the data, handling missing values, cleaning, and transforming it.

### Examining data

We start by loading our datasets and inspecting them to get a better sense of their structure. 

In [ ]:
news2023 = pd.read_csv("news2023.csv", names=["date", "title", "desc", "type", "link"]) # import csv with column names
news2024 = pd.read_csv("news2024.csv", names=["date", "title", "desc", "type", "link"])

In [ ]:
news2023

In [ ]:
news2024.head()

In [ ]:
len(news2024) # check the number of rows

In [ ]:
news2024.shape # check the number of rows and columns

In [ ]:
news2024.info() # summarize by the columns

In [ ]:
news2024.describe() # useful for numerical data

In [ ]:
type(news2024.date[0]) # check the data type

In [ ]:
news2023.type.unique() # check the unique values of a column

In [ ]:
news2023.type.value_counts() # check each value's number of occurrences

### Handling missing values

In [ ]:
news2024.isnull().sum() # identify missing data

In [ ]:
news2023.isnull().sum()

In [ ]:
news2023[news2023.title.isnull()] # pinpoint which rows have missing values

In [ ]:
news2023[news2023.link.isnull()] # check again by "link"

Missing values can often skew analyses or create errors during analysis. Here are some common strategies for handling missing values:

- Manually review and fix the missing data (if possible)
- Remove missing data e.g., drop rows
- Impute missing data e.g., replace with the mean (for numerical data)
- Fill with a specific value e.g., replace with "unknown" or "NA"
- Predictive imputation e.g., use machine learning models to predict missing values based on other features
- Leave as missing

For our example, we choose the first option because we have access to the original data source. Let's go back to the previous error messages.

&#x1F4A1; **Markdown**: Markdown is a markup language. It can be used as a text-to-HTML conversion tool. Read more on Jupyter [Markdown cells](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html). It can help us read the error messages and fix the missing values. Below is a direct copy & paste of the [error messages](#error) (in a Markedown cell).

https://transport.ec.europa.eu/news-events/news_en?page=7
https://transport.ec.europa.eu/news-events/news_en?page=8
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-12-12T12:00:00Z">12 December 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.wbif.eu/news-details/transport-ministers-discuss-reforms-within-western-balkans-transport-community-summit"><span class="ecl-link__label">Transport Ministers discuss reforms within Western Balkans at Transport Community Summit</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>Transport ministers from the Western Balkans, Georgia, Moldova, and Ukraine, as well as the European Commission, gathered at the annual Ministerial Council Meeting of the Transport Community in Skopje, North Macedonia, on 12 December 2023.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-12-12T12:00:00Z">12 December 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.wbif.eu/news-details/transport-ministers-discuss-reforms-within-western-balkans-transport-community-summit"><span class="ecl-link__label">Transport Ministers discuss reforms within Western Balkans at Transport Community Summit</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>Transport ministers from the Western Balkans, Georgia, Moldova, and Ukraine, as well as the European Commission, gathered at the annual Ministerial Council Meeting of the Transport Community in Skopje, North Macedonia, on 12 December 2023.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=9
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-10-25T12:00:00Z">25 October 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.ecac-ceac.org/news/960-press-release-european-aviation-leaders-unite-for-holistic-sustainability-in-the-ecac-eu-dialogue"><span class="ecl-link__label">European Commission joins the 12th edition of the ECAC/EU Dialogue</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>European aviation leaders have come together in Valencia for the 12th edition of the ECAC/EU Dialogue, reaffirming their commitment to sustainable aviation.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-10-25T12:00:00Z">25 October 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.ecac-ceac.org/news/960-press-release-european-aviation-leaders-unite-for-holistic-sustainability-in-the-ecac-eu-dialogue"><span class="ecl-link__label">European Commission joins the 12th edition of the ECAC/EU Dialogue</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>European aviation leaders have come together in Valencia for the 12th edition of the ECAC/EU Dialogue, reaffirming their commitment to sustainable aviation.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=10
https://transport.ec.europa.eu/news-events/news_en?page=11
https://transport.ec.europa.eu/news-events/news_en?page=12
https://transport.ec.europa.eu/news-events/news_en?page=13
https://transport.ec.europa.eu/news-events/news_en?page=14
https://transport.ec.europa.eu/news-events/news_en?page=15
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-03-14T12:00:00Z">14 March 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.eib.org/en/press/all/2023-108-investeu-eur3-4-billion-to-modernise-the-palermo-catania-railway-line"><span class="ecl-link__label">Italy: InvestEU - €3.4 billion to modernise the Palermo-Catania railway line</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The modernisation of 178 km of the Palermo-Catania line will reduce current travel times by a third, linking the two cities with a direct two-hour rail service, which will have a significant impact on economic, social, and sustainable development in Sicily.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-03-14T12:00:00Z">14 March 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://www.eib.org/en/press/all/2023-108-investeu-eur3-4-billion-to-modernise-the-palermo-catania-railway-line"><span class="ecl-link__label">Italy: InvestEU - €3.4 billion to modernise the Palermo-Catania railway line</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The modernisation of 178 km of the Palermo-Catania line will reduce current travel times by a third, linking the two cities with a direct two-hour rail service, which will have a significant impact on economic, social, and sustainable development in Sicily.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting title: 'NoneType' object has no attribute 'get_text'
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-02-28T12:00:00Z">28 February 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://mobilityweek.eu/media-corner/"><span class="ecl-link__label">Braga, Sofia and Zagreb among the finalists for European urban mobility awards</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The finalists for the EUROPEANMOBILITYWEEK award 2022 and the first-ever MOBILITYACTION award have been announced.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
Error extracting link: 'NoneType' object is not subscriptable
<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2023-02-28T12:00:00Z">28 February 2023</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://mobilityweek.eu/media-corner/"><span class="ecl-link__label">Braga, Sofia and Zagreb among the finalists for European urban mobility awards</span><svg aria-hidden="false" class="ecl-icon ecl-icon--2xs ecl-link__icon" focusable="false"><use xlink:href="/themes/contrib/oe_theme/dist/ec/images/icons/sprites/icons.svg#external"></use></svg></a></div><div class="ecl-content-block__description"><p>The finalists for the EUROPEANMOBILITYWEEK award 2022 and the first-ever MOBILITYACTION award have been announced.</p></div><div class="ecl-content-block__list-container"></div></div></article>
====
https://transport.ec.europa.eu/news-events/news_en?page=16
https://transport.ec.europa.eu/news-events/news_en?page=17

In [ ]:
news2023.iloc[10].title # the first missing title

In [ ]:
# write the title to the cell
news2023.iloc[10].title = "Transport Ministers discuss reforms within Western Balkans at Transport Community Summit"

In [ ]:
news2023[news2023.title.isnull()] # check missing titles again. the above one is no longer there

In [ ]:
# do the same for the missing link
news2023.iloc[10].link = "https://www.wbif.eu/news-details/transport-ministers-discuss-reforms-within-western-balkans-transport-community-summit"

In [ ]:
# do the same for the rest missing values
news2023.iloc[21].title = "European Commission joins the 12th edition of the ECAC/EU Dialogue"
news2023.iloc[21].link = "https://www.ecac-ceac.org/news/960-press-release-european-aviation-leaders-unite-for-holistic-sustainability-in-the-ecac-eu-dialogue"
news2023.iloc[73].title = "Italy: InvestEU - €3.4 billion to modernise the Palermo-Catania railway line"
news2023.iloc[73].link = "https://www.eib.org/en/press/all/2023-108-investeu-eur3-4-billion-to-modernise-the-palermo-catania-railway-line"
news2023.iloc[77].title = "Braga, Sofia and Zagreb among the finalists for European urban mobility awards"
news2023.iloc[77].link = "https://mobilityweek.eu/media-corner/"

In [ ]:
news2023.isnull().sum() # check again. no more missing values

### Merging datasets

There are many different ways of merging datasets. Check this [documentation](https://pandas.pydata.org/docs/user_guide/merging.html) or ask ChatGPT. Let's combine our two datasets into one.

In [ ]:
news = pd.concat([news2023, news2024], ignore_index=True) # concatenate and re-index

In [ ]:
news

In [ ]:
news.isnull().sum()

Nothing is missing. However, we might have created duplicated values when merging datasets.

### Cleaning

In [ ]:
news[news.duplicated()] # identify duplicated rows

In [ ]:
news.duplicated() # returns a Boolean Series indicating whether each row is a duplicate of a previous row

In [ ]:
news.duplicated().tail(11)

In [ ]:
news = news.drop_duplicates() # drop duplicated rows

In [ ]:
news

We also want to fix the date using [`pandas.to_datetime` function](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html). 

In [ ]:
type(news.date[0])

In [ ]:
news.loc[:, "date"] = pd.to_datetime(news.date) # change the data type to date

In [ ]:
type(news.date[0])

In [ ]:
news

In [ ]:
news = news.sort_values(by="date", ascending=False) # sort the rows by date

In [ ]:
news

In [ ]:
news.head(10)

In [ ]:
news['date'] = pd.to_datetime(news['date']) # make sure the 'date' column is of datetime type
news = news[news['date'].dt.year!=2022] # remove news from 2022

In [ ]:
news

In [ ]:
news.type.unique()

In [ ]:
news[news.type=="Press release"]

In [ ]:
news[news.type=="Supplementary information"]

Let's only keep the news.

In [ ]:
news = news[news.type=="News article"]

In [ ]:
news = news.reset_index(drop=True) # reset the index

In [ ]:
news = news[["date", "title", "desc"]] # new df with selected columns

In [ ]:
news

In [ ]:
news.to_csv("news_clean.csv", index=False) # save to csv

&#x2753;**<font color=cornflowerblue>QUESTION: </font>** What are the common tasks involved in data wrangling?

&#x270A; **<font color=firebrick>DO THIS: </font>** Here are two options to practice what we've learned and to further develop your data skills:

1. Do some data wrangling on the data you scraped last week.
2. Explore the existing `news_clean.csv` dataset. What research question can it answer? Using what methods/tools?

Feel free to pick one task or do both. Have fun!

In [ ]:
# put your code here













---------
### Congratulations, we are done!

This notebook is written by [Meng Cai](https://www.verkehr.tu-darmstadt.de/vv/das_institut_ivv/team_ivv/wissenschaftliche_mitarbeiter_doktoranden/meng_cai/standardseite_204.de.jsp), Technical University of Darmstadt. This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a>